## Topic modeling appliqué aux articles de journaux - classification et résumé pour la supervision de sujets

La catégorisation automatiques de textes est un enjeu clé pour la veille médiatique et la classification thématique des textes. Cependant, les méthodes traditionnelles de classification supervisée nécessitent un jeu de données conséquent, annoté et spécifique à un domaine.

Ne disposant pas d'un tel jeu de données, le Zero-shot learning (ZSL) est une méthode commune pour outrepasser le manque de données. Cette méthode prédit une classe quelle n'a pas vu lors de son entrainement, en tirant parti du transfer learning d'un modèle pré-entrainé.

Pour ce projet, je propose une solution de topic modeling basé sur le ZSL, en exploitant les capacités du modèle mDeBERTa finetuné pour le natural language inference multi-langue (mNLI), qui évalue la compatibilité sémantique entre un texte et chaque label candidat, permettant ainsi une classification sans données d’entraînement spécifiques.

Le Topic Modeling est évalué sur le dataset Obsinfox ([Icard B. et al 2024](https://arxiv.org/abs/2403.16099v1)), un corpus de 100 articles français ayant une faible crédibilité (Fake News, désinfomation, mésinformation). Ce dataset permet d'évaluer le modèle sur des textes ambigus ou hors contexte, reflétant des conditions réelles de veille médiatique.".

Les données ont été labélé manuellement pour évaluer le modèle. Le topic modeling a un score F1 de 0.59. Les résultats sont comparable à[Boyina K. et al 2024](https://ieeexplore.ieee.org/document/10724558), qui rapporte un score F1 de 0.58 pour la classification d'articles de journaux avec mBERT.

Ce score démontre la faisabilité de la classification zero-shot pour des textes hors contexte et ouvre la voie à des application rapide et adaptable pour la supervision de média ou la détection de désinformation.

In [ ]:
import pandas as pd

df = pd.read_csv('obsinfox.csv')
df = df[['URL', 'Title']].drop_duplicates()
df.reset_index(names='ID', inplace=True)

### 1) Scraper les articles à partir d'URL

Définition de deux fonctions, pour:
- scraper l'article à partir d'une URL
- process l'article

In [ ]:
import re
from newspaper import Article

# Mettre en forme le texte et ne récupérer que les 5 premières phrases
def process_article(article: str, nb_phrase = 5) -> str:
    article = re.sub(r'[\n\t\r]+', ' ', article)
    article = re.sub(r' +', ' ', article)
    article = re.sub(r"\"", "'", article)
    article = article.strip()
    article = re.split(r'(?<=[.!?]) +', article)
    article = [phrase for phrase in article if len(phrase)]
    return ' '.join(article[:nb_phrase])

# Récupérer et traiter l'article à partir de l'URL
def get_article(url: str) -> str:
    try:
        article = Article(url)
        article.download()
        article.parse()
        if article.title:
            article_title = article.title
            article = article_title + '. ' + article.text
        else:   
            article = article.text
        return process_article(article), article_title
    except:
        return None
        

On teste sur le premier lien:

In [ ]:
url = df['URL'][0]
article, title = get_article(url)
print("Title:", title, "\nArticle:", article)

Title: La relation entre la technologie et la religion 
Article: La relation entre la technologie et la religion. Par Austin Cline − Le 27 fevrier 2019 − Source Learn Religions De nombreux laïcs et non-croyants de toutes sortes ont tendance à considérer la religion et la science comme fondamentalement incompatibles. Cette incompatibilité est également imaginée pour s’étendre à la relation entre la religion et la technologie, puisque la technologie est un produit de la science et que la science ne peut pas aller de l’avant sans la technologie, surtout aujourd’hui. Ainsi, bon nombre d’athées s’émerveillent avec incrédulité du nombre d’ingénieurs qui sont également des créationnistes et du nombre de personnes dans les industries de haute technologie qui affichent de grandes motivations religieuses. Mélanger technologie et religion Pourquoi assistons-nous à un enchantement généralisé pour la technologie et en même temps à une résurgence mondiale du fondamentalisme religieux ?


Application sur notre données :

In [ ]:
df[['Text' , 'Title']] = df['URL'].apply(lambda url: pd.Series(get_article(url)))
df.head()

In [ ]:
df = df[ df['Text'].notnull() ]
df.head()

,ID,URL,Title,Text
0,0,https://lesakerfrancophone.fr/la-relation-entr...,La relation entre la technologie et la religion,La relation entre la technologie et la religio...
1,1,https://www.breizh-info.com/2021/01/27/157958/...,"Confinement. Les habitants de Brest, Morlaix e...","Confinement. Les habitants de Brest, Morlaix e..."
4,4,https://lesakerfrancophone.fr/selon-ubs-les-pr...,"Selon UBS, les « propriétés d’assurance tant d...","Selon UBS, les « propriétés d’assurance tant d..."
6,6,https://www.profession-gendarme.com/essonne-le...,Essonne : Les voleurs tombent sur des membres ...,Essonne : Les voleurs tombent sur des membres ...
7,7,https://www.dreuz.info/2021/10/la-misere-energ...,"La misère énergétique, pour punir les consomma...","La misère énergétique, pour punir les consomma..."


### 2) Résumer le texte

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, pipeline

path = 'models/bart-large-cnn'

summarizer = pipeline("summarization", 
                      model= BartForConditionalGeneration.from_pretrained(path),
                      tokenizer=BartTokenizer.from_pretrained(path))

def get_summary(text:str) -> str:
    summary = summarizer(text, max_length=120, min_length=30, do_sample=False)
    return summary[0]['summary_text']

Device set to use cpu


Application sur les articles:

In [ ]:
df['Summary'] = df['Text'].apply(get_summary)
df.head()

Your max_length is set to 120, but your input_length is only 104. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)
Your max_length is set to 120, but your input_length is only 115. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Your max_length is set to 120, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Your max_length is set to 120, but your input_length is only 111. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)


,ID,URL,Title,Text,Summary
0,0,https://lesakerfrancophone.fr/la-relation-entr...,La relation entre la technologie et la religion,La relation entre la technologie et la religio...,De nombreux laïcs et non-croyants de toutes so...
1,1,https://www.breizh-info.com/2021/01/27/157958/...,"Confinement. Les habitants de Brest, Morlaix e...","Confinement. Les habitants de Brest, Morlaix e...",Dry January is opération commerciale américain...
4,4,https://lesakerfrancophone.fr/selon-ubs-les-pr...,"Selon UBS, les « propriétés d’assurance tant d...","Selon UBS, les « propriétés d’assurance tant d...",UBS: Les propriétés d’assurance tant de fois é...
6,6,https://www.profession-gendarme.com/essonne-le...,Essonne : Les voleurs tombent sur des membres ...,Essonne : Les voleurs tombent sur des membres ...,Quatre hommes interpellés à Guillerval (Essonn...
7,7,https://www.dreuz.info/2021/10/la-misere-energ...,"La misère énergétique, pour punir les consomma...","La misère énergétique, pour punir les consomma...",On se souvient that le célèbre gauchiste améri...


In [ ]:
df.to_csv('obsinfox_v1.csv', index=False)

### 3) Topic Modeling

In [ ]:
from typing import Union, List
from transformers import AutoTokenizer, pipeline
import torch

OUT_DIR = "models/mDeBERTa"
device = 0 if torch.cuda.is_available() else -1

# Liste des topics pour la classification zéro-shot
zeroshot_topic_list = [
"Politique",
"Économie & finance",
"Santé publique & médecine",
"Sécurité & criminalité",
"Environnement & climat",
"Science & technologie",
"Société & culture",
]

# recharger le tokenizer en appliquant le fix
tokenizer = AutoTokenizer.from_pretrained(OUT_DIR, fix_mistral_regex=True)

classifier = pipeline(
    task="zero-shot-classification",
    model=OUT_DIR,
    tokenizer=tokenizer,
    device=device
)
def get_topic(input: Union[str, List[str]]) -> Union[str, List[str]]:
    result = classifier(
    sequences=input,
    candidate_labels=zeroshot_topic_list,
    hypothesis_template="Ce texte parle de {}."
    )
    if isinstance(input, list):
        return [res['labels'][0] for res in result]
    elif isinstance(input, str):
        return result['labels'][0]
    else:
        return None

d:\PROJET\Obsinfox\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [ ]:
topics = get_topic(df['Summary'].tolist())
df['Topic'] = topics
df.head()

,ID,URL,Title,Text,Summary,Topic
0,0,https://lesakerfrancophone.fr/la-relation-entr...,La relation entre la technologie et la religion,La relation entre la technologie et la religio...,De nombreux laïcs et non-croyants de toutes so...,Science & technologie
1,1,https://www.breizh-info.com/2021/01/27/157958/...,"Confinement. Les habitants de Brest, Morlaix e...","Confinement. Les habitants de Brest, Morlaix e...",Dry January is opération commerciale américain...,Société & culture
4,4,https://lesakerfrancophone.fr/selon-ubs-les-pr...,"Selon UBS, les « propriétés d’assurance tant d...","Selon UBS, les « propriétés d’assurance tant d...",UBS: Les propriétés d’assurance tant de fois é...,Sécurité & criminalité
6,6,https://www.profession-gendarme.com/essonne-le...,Essonne : Les voleurs tombent sur des membres ...,Essonne : Les voleurs tombent sur des membres ...,Quatre hommes interpellés à Guillerval (Essonn...,Sécurité & criminalité
7,7,https://www.dreuz.info/2021/10/la-misere-energ...,"La misère énergétique, pour punir les consomma...","La misère énergétique, pour punir les consomma...",On se souvient that le célèbre gauchiste améri...,Environnement & climat


In [ ]:
df.to_csv('obsinfox_v2.csv', index=False)

### 4) Evaluation du topic modeling

In [1]:
import pandas as pd
from sklearn.metrics import f1_score

df = pd.read_csv('obsinfox_v2.csv')

y_true = df['Label']
y_pred = df['Topic']

f1 = f1_score(y_true, y_pred, average='weighted')
print("F1-score:", f1)

F1-score: 0.5912229936620181
